# 基本分类模型
:label:`sec_classification`

你可能已经注意到，从零开始的实现和使用框架功能的简洁实现，在回归的情况下非常相似。对于分类也是如此。由于本书中的许多模型都涉及分类，因此值得添加功能来专门支持这种设置。本节提供了一个分类模型的基类，以简化未来的代码。

In [1]:
import torch
from d2l import torch as d2l

## `分类器` 类

我们在下面定义了`Classifier`类。在`validation_step`中，我们报告验证批次上的损失值和分类准确率。每`num_val_batches`个批次后我们进行一次更新。这样做可以生成整个验证数据的平均损失和准确率。如果最后一个批次包含的样本较少，这些平均数值可能不完全准确，但我们为了保持代码简洁而忽略了这一微小差异。

In [2]:
class Classifier(d2l.Module):  #@save
    """The base class of classification models."""
    def validation_step(self, batch):
        Y_hat = self(*batch[:-1])
        self.plot('loss', self.loss(Y_hat, batch[-1]), train=False)
        self.plot('acc', self.accuracy(Y_hat, batch[-1]), train=False)

默认情况下，我们使用随机梯度下降优化器，对小批量数据进行操作，就像我们在线性回归的背景下所做的那样。

In [3]:
@d2l.add_to_class(d2l.Module)  #@save
def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=self.lr)

## 准确率

给定预测的概率分布 `y_hat`，
我们通常会选择具有最高预测概率的类别
作为硬性预测结果。
实际上，许多应用要求我们必须做出选择。
例如，Gmail 必须将电子邮件分类为“主要”、“社交”、“更新”、“论坛”或“垃圾邮件”。
它可能会在内部估计概率，
但最终必须从这些类别中选择一个。

当预测与标签类 `y` 一致时，它们是正确的。
分类准确率是所有正确预测所占的比例。
尽管直接优化准确率可能很困难（它是不可微的），
但它通常是我们在乎的最重要的性能指标。它往往是基准测试中的相关量。因此，我们在训练分类器时几乎总是会报告它。

准确率的计算方法如下。
首先，如果 `y_hat` 是一个矩阵，
我们假设第二维度存储了每个类别的预测分数。
我们使用 `argmax` 来获取每行最大值的索引作为预测类别。
然后我们逐元素比较预测类别和真实标签 `y`。
由于等号运算符 `==` 对数据类型敏感，
我们将 `y_hat` 的数据类型转换为与 `y` 相同。
结果是一个包含 0（假）和 1（真）的张量。
求和后得到正确预测的数量。

In [4]:
@d2l.add_to_class(Classifier)  #@save
def accuracy(self, Y_hat, Y, averaged=True):
    """Compute the number of correct predictions."""
    Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
    preds = Y_hat.argmax(axis=1).type(Y.dtype)
    compare = (preds == Y.reshape(-1)).type(torch.float32)
    return compare.mean() if averaged else compare

## 摘要

分类是一个非常常见的问题，以至于它值得拥有自己的便捷函数。在分类中至关重要的是分类器的*准确性*。请注意，虽然我们通常主要关心准确性，但出于统计和计算的原因，我们会训练分类器以优化各种其他目标。然而，无论在训练过程中最小化了哪种损失函数，拥有一种便捷的方法来实证评估我们分类器的准确性都是非常有用的。

## 练习

1. 用 $L_\textrm{v}$ 表示验证损失，并设 $L_\textrm{v}^\textrm{q}$ 为本节中通过损失函数平均得到的快速且粗略的估计值。最后，用 $l_\textrm{v}^\textrm{b}$ 表示最后一个迷你批次上的损失。用 $L_\textrm{v}^\textrm{q}$、$l_\textrm{v}^\textrm{b}$ 以及样本和迷你批次的大小来表示 $L_\textrm{v}$。
1. 证明快速且粗略的估计值 $L_\textrm{v}^\textrm{q}$ 是无偏的。也就是说，证明 $E[L_\textrm{v}] = E[L_\textrm{v}^\textrm{q}]$。为什么你仍然希望使用 $L_\textrm{v}$？
1. 给定一个多类分类损失，用 $l(y,y')$ 表示当我们看到 $y$ 时估计 $y'$ 的惩罚，并给定一个概率 $p(y \mid x)$，制定选择 $y'$ 的最优规则。提示：使用 $l$ 和 $p(y \mid x)$ 表达预期损失。

[讨论](https://discuss.d2l.ai/t/6809)